The sparksession instance is the way Spark executes user-defined manipulation across the cluster

In [4]:
# start a spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## DBFS (Databrick File System)

More detail can be found [here](https://docs.databricks.com/data/databricks-file-system.html)

Type dbutils.fs.help() for help.

In [5]:
dbutils.fs.help()

NameError: name 'dbutils' is not defined

In [0]:
# List current directory
dbutils.fs.ls('/')

Notebooks support a shorthand—%fs magic commands—for accessing the dbutils filesystem module. Most dbutils.fs commands are available using %fs magic commands

In [0]:
# List current directory


#### TODO #1: Create a new directory called `data1050` and show where it locates

## An End to End example
We will use spark to analyze some flight data from the United States Bureau of Transporation statistics

Data in this example is located at this directory: /databricks-datasets/definitive-guide/data

You can also download the data and all notebooks of the text on Gitbub https://github.com/databricks/Spark-the-Definitive-guide

##### Read data into Spark DataFrame

In [6]:
# read 2015 summary data
path= "/databricks-datasets/definitive-guide"
flightData2015 = spark\
  .read\
  .option("inferSchema", "false")\
  .option("header", "false")\
  .csv(path+"/data/flight-data/csv/2015-summary.csv")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv.

In [7]:
df = spark.read.csv(path+"/data/flight-data/csv/2015-summary.csv", header=True, inferSchema=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv.

In [0]:
# take a look at our schema


In [0]:
df

##### Read data into SQL Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS flightData2015
USING CSV
OPTIONS (path '/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv', header 'true', inferSchema 'true');

In [0]:
%sql
SELECT * FROM flightData2015 LIMIT 10;

In [0]:
# covert the DataFrame to a table so that we can use sql to query the table
df.createOrReplaceTempView("flight_data_2015")

##### SQL Table vs. Spark DataFrame

In [0]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY count(1) DESC
""")

sqlWay.show()

In [0]:
dataFrameWay = df\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()\
  .orderBy("count", ascending=False)

dataFrameWay.show()

In [0]:
sqlWay.explain()
dataFrameWay.explain()

##### PySpark Functions

In [0]:
from pyspark.sql.functions import *

#### TODO #2: 